In [1]:
from datasets import load_dataset, load_metric

In [2]:
metric = load_metric("rouge")

In [21]:
data_files = {"train": "summarization/train.jsonl"}
split = {"train":"train[:80%]", "valid":"train[80%:]"}
raw_datasets = load_dataset("json", data_files=data_files, split=split)

Using custom data configuration default-fbff2080bc386d28
Reusing dataset json (C:\Users\jinma\.cache\huggingface\datasets\json\default-fbff2080bc386d28\0.0.0\83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02)


In [23]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['media', 'id', 'article_original', 'abstractive', 'extractive'],
        num_rows: 34242
    })
    valid: Dataset({
        features: ['media', 'id', 'article_original', 'abstractive', 'extractive'],
        num_rows: 8561
    })
})

In [24]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [25]:
show_random_elements(raw_datasets["train"], num_examples=2)

,media,id,article_original,abstractive,extractive
0,대전일보,332585287,"[[보령] 보령시가 중소기업 경쟁력을 강화하기 위해 추진해 온 산·학·관 협력기술개발 지원사업이 큰 결실을 맺었다., 시는 29일 오전 보령엔지니어링에서 김인영 대표와 미얀마 조달청 구매담당 과장 등 관계자가 참석한 가운데 미얀마와 자동포구 청소기에 대한 200억 원대 수출계약을 체결했다고 밝혔다., 이날 수출계약은 지난해 8월 미얀마 조달청에서 추진한 샘플 입찰전에서 미국과 중국, 프랑스 등 11개의 유명한 업체들과 경쟁에서 따냈다., 우수한 제품 성능을 국제적으로 인정받게 된 것이다., 이번 협약 이후 105mm와 155mm 자동포구 청소기 각 10세트를 오는 4월에는 기갑부대 2종 각 10세트를 납품할 예정이다., 또한 지난해 7월에는 방글라데시와 샘플계약으로 오는 2020년까지 200억 원 규모의 수출 성과를 이뤘고, 올해 5월 이후에는 터키와 500억 원 대 수출계약을 체결할 것으로 예상되고 있으며 파키스탄과 사우디아라비아, 말레이시아, 태국, 이집트, 베트남 등 6개국과도 수출 교섭중에 있다., 특히 이번 수출계약은 그동안 보령시가 자금사정 등으로 어려움을 겪고 있는 지역 중소 제조업체에 아주자동차대학의 우수한 전문기술을 활용해 기업의 제품개발, 공정개선 등 애로사항 해결을 통한 경쟁력 강화를 위해 추진해 온 산., 학.관 협력 기술개발이 밑바탕이 돼 더욱 주목받고 있다., 이번에 보령엔지니어링이 대표 제품으로 내놓은 자동포구 청소기는 지난 2017년 보령시 산·학·관 협력 기술개발 사업으로 개발됐으며 청소 속도, 회전 속도 향상으로 청소시간을 획기적으로 단축하고, 최적화된 설계를 통한 원가절감으로 가격경쟁력 또한 우수해 특허 등록도 완료했다., 아울러 지속되는 수출 협약에 따라 시와 보령엔지니어링은 현재 15명의 지역 고용 채용 규모를 오는 2020년까지 50명으로 3배 이상 늘리는 등 기업의 성장과 함께 지역과 상생하는 사회적 가치를 실현해 나갈 것으로 기대를 모으고 있다., 김동일 시장은 ""전국 제일의 기업하기 좋은 도시로 만들어 나가기 위해 기업 안정 자금은 물론 기술개발을 지원해 온 것이 큰 결실을 맺게 됐다""며 ""시는 앞으로도 중소기업의 경쟁력 강화와 해외수출 확대 지원으로 지역경제를 활성화하기 위해 최선을 다하겠다""고 말했다., 보령엔지니어링은 육·해·공군의 전투장비 및 무기체계 핵심부품을 생산하는 회사로 국가산업 발전 및 전력증강 사업에 기여하는데 역점을 두고 회사를 운영해오고 있으며, 지난해에는 연매출 30억 원을 올렸다.]",29일 보령 엔지니어링이 미얀마와 자동포구 청소기에 대한 200억 원대 수출계약 체결은 보령시의 산·학·관 협력 기술개발 지원사업이 큰 결실을 맺은 것으로써 이 청소기는 지난 2017년 보령시 산·학·관 협력 기술개발 사업이 바탕을 이루어 개발된 제품이다.,"[0, 1, 8]"
1,대구일보,346689275,"[대구시는 29일 시청 별관에서 빅데이터 통합 플랫폼 구축 중간보고회를 개최한다., 이는 지난해 5월 행정안전부의 지역 빅데이터 허브 사업 공모에서 대구시가 선정돼 추진하는 것이다., 빅데이터 통합플랫폼은 여기저기 흩어져 있던 데이터를 체계적으로 수집·저장하고, 수집된 데이터를 검색·활용할 수 있는 데이터셋과 데이터 맵을 구축하는 사업이다., 국비 5억 원 등 총사업비 10억 원을 투입한다., 대구시와 구·군청이 보유한 공공데이터 775건을 개방하고 1만4천여 건의 통계데이터를 제공한다., 지역개발, 문화관광, 과학기술 등 12개 카테고리로 데이터를 분류하고 주요 통계데이터나 인기·최신 데이터를 앞쪽에 둬 이용 편의를 높인다., 전기차 충전 인프라 입지분석, 우리 동네 사업 분석 등 그동안 시에서 수행한 빅데이터 분석 사례와 관련 데이터도 다운 받을 수 있다., 데이터는 파일이나 응용프로그램 인터페이스(API) 형태로 제공해 누구나 쉽게 이용할 수 있다., 대구시는 빅데이터 통합플랫폼 구축이 완료되면 빅데이터 활용센터와 연계해 행정 내부는 물론 학생, 창업자 등 시민 누구나 대구시 공공데이터를 활용, 분석할 수 있는 환경도 제공한다., 배춘식 대구시 데이터통계담당관은 “올해를 대구시 데이터 기반 행정의 원년으로 삼고 데이터 행정 추진을 위한 조직 신설, 로드맵 마련, 전문 인력 충원, 직원교육 실시 등을 추진하고 있다”고 설명했다.]","대구시는 빅데이터 플랫폼을 구축하여 데이터를 인터페이스 형태로 제공해 시민 누구나 대구시 공공데이터를 활용, 분석할 수 있는 환경을 제공한다.","[7, 8, 4]"


In [28]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

In [29]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained("hyunwoongko/kobart")

In [30]:
tokenizer("Hello, this one sentence!")

{'input_ids': [271, 16997, 20858, 243, 19133, 15868, 1700, 14889, 300, 18482, 16884, 15265, 22038, 232], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [31]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[271, 16997, 20858, 243, 19133, 15868, 1700, 14889, 300, 18482, 16884, 15265, 22038, 232], [283, 303, 15868, 1700, 15868, 1700, 15195, 310, 16160, 14879, 18482, 16884, 15265, 22038, 245]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [32]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[271, 16997, 20858, 243, 19133, 15868, 1700, 14889, 300, 18482, 16884, 15265, 22038, 232], [283, 303, 15868, 1700, 15868, 1700, 15195, 310, 16160, 14879, 18482, 16884, 15265, 22038, 245]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [33]:
prefix = ""

In [38]:
raw_datasets["train"]

Dataset({
    features: ['media', 'id', 'article_original', 'abstractive', 'extractive'],
    num_rows: 34242
})

In [47]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + "".join(doc) for doc in examples["article_original"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["abstractive"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [48]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [49]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['abstractive', 'article_original', 'attention_mask', 'extractive', 'id', 'input_ids', 'labels', 'media'],
        num_rows: 34242
    })
    valid: Dataset({
        features: ['abstractive', 'article_original', 'attention_mask', 'extractive', 'id', 'input_ids', 'labels', 'media'],
        num_rows: 8561
    })
})

In [54]:
from transformers import (
    AutoModelForSeq2SeqLM, 
    DataCollatorForSeq2Seq, 
    Seq2SeqTrainingArguments, 
    Seq2SeqTrainer
)

model = AutoModelForSeq2SeqLM.from_pretrained("hyunwoongko/kobart")

In [59]:
batch_size = 2

args = Seq2SeqTrainingArguments(
    "test-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
)

In [60]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [61]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [63]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [64]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, article_original, media, extractive, abstractive.
***** Running training *****
  Num examples = 34242
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 51363


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.390800,1.380210,29.550200,9.444000,29.019400,28.992100,20.000000
2,1.166000,1.361020,28.739800,9.239700,28.227400,28.188700,20.000000
3,0.978000,1.379811,29.919400,9.617100,29.352800,29.339000,20.000000


Saving model checkpoint to test-summarization\checkpoint-500
Configuration saved in test-summarization\checkpoint-500\config.json
Model weights saved in test-summarization\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test-summarization\checkpoint-500\tokenizer_config.json
Special tokens file saved in test-summarization\checkpoint-500\special_tokens_map.json
Saving model checkpoint to test-summarization\checkpoint-1000
Configuration saved in test-summarization\checkpoint-1000\config.json
Model weights saved in test-summarization\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in test-summarization\checkpoint-1000\tokenizer_config.json
Special tokens file saved in test-summarization\checkpoint-1000\special_tokens_map.json
Saving model checkpoint to test-summarization\checkpoint-1500
Configuration saved in test-summarization\checkpoint-1500\config.json
Model weights saved in test-summarization\checkpoint-1500\pytorch_model.bin
tokenizer config file saved i

Deleting older checkpoint [test-summarization\checkpoint-7500] due to args.save_total_limit
Saving model checkpoint to test-summarization\checkpoint-9500
Configuration saved in test-summarization\checkpoint-9500\config.json
Model weights saved in test-summarization\checkpoint-9500\pytorch_model.bin
tokenizer config file saved in test-summarization\checkpoint-9500\tokenizer_config.json
Special tokens file saved in test-summarization\checkpoint-9500\special_tokens_map.json
Deleting older checkpoint [test-summarization\checkpoint-8000] due to args.save_total_limit
Saving model checkpoint to test-summarization\checkpoint-10000
Configuration saved in test-summarization\checkpoint-10000\config.json
Model weights saved in test-summarization\checkpoint-10000\pytorch_model.bin
tokenizer config file saved in test-summarization\checkpoint-10000\tokenizer_config.json
Special tokens file saved in test-summarization\checkpoint-10000\special_tokens_map.json
Deleting older checkpoint [test-summarizati

Model weights saved in test-summarization\checkpoint-17500\pytorch_model.bin
tokenizer config file saved in test-summarization\checkpoint-17500\tokenizer_config.json
Special tokens file saved in test-summarization\checkpoint-17500\special_tokens_map.json
Deleting older checkpoint [test-summarization\checkpoint-16000] due to args.save_total_limit
Saving model checkpoint to test-summarization\checkpoint-18000
Configuration saved in test-summarization\checkpoint-18000\config.json
Model weights saved in test-summarization\checkpoint-18000\pytorch_model.bin
tokenizer config file saved in test-summarization\checkpoint-18000\tokenizer_config.json
Special tokens file saved in test-summarization\checkpoint-18000\special_tokens_map.json
Deleting older checkpoint [test-summarization\checkpoint-16500] due to args.save_total_limit
Saving model checkpoint to test-summarization\checkpoint-18500
Configuration saved in test-summarization\checkpoint-18500\config.json
Model weights saved in test-summariz

Model weights saved in test-summarization\checkpoint-26000\pytorch_model.bin
tokenizer config file saved in test-summarization\checkpoint-26000\tokenizer_config.json
Special tokens file saved in test-summarization\checkpoint-26000\special_tokens_map.json
Deleting older checkpoint [test-summarization\checkpoint-24500] due to args.save_total_limit
Saving model checkpoint to test-summarization\checkpoint-26500
Configuration saved in test-summarization\checkpoint-26500\config.json
Model weights saved in test-summarization\checkpoint-26500\pytorch_model.bin
tokenizer config file saved in test-summarization\checkpoint-26500\tokenizer_config.json
Special tokens file saved in test-summarization\checkpoint-26500\special_tokens_map.json
Deleting older checkpoint [test-summarization\checkpoint-25000] due to args.save_total_limit
Saving model checkpoint to test-summarization\checkpoint-27000
Configuration saved in test-summarization\checkpoint-27000\config.json
Model weights saved in test-summariz

***** Running Evaluation *****
  Num examples = 8561
  Batch size = 2
Saving model checkpoint to test-summarization\checkpoint-34500
Configuration saved in test-summarization\checkpoint-34500\config.json
Model weights saved in test-summarization\checkpoint-34500\pytorch_model.bin
tokenizer config file saved in test-summarization\checkpoint-34500\tokenizer_config.json
Special tokens file saved in test-summarization\checkpoint-34500\special_tokens_map.json
Deleting older checkpoint [test-summarization\checkpoint-33000] due to args.save_total_limit
Saving model checkpoint to test-summarization\checkpoint-35000
Configuration saved in test-summarization\checkpoint-35000\config.json
Model weights saved in test-summarization\checkpoint-35000\pytorch_model.bin
tokenizer config file saved in test-summarization\checkpoint-35000\tokenizer_config.json
Special tokens file saved in test-summarization\checkpoint-35000\special_tokens_map.json
Deleting older checkpoint [test-summarization\checkpoint-33

Saving model checkpoint to test-summarization\checkpoint-43000
Configuration saved in test-summarization\checkpoint-43000\config.json
Model weights saved in test-summarization\checkpoint-43000\pytorch_model.bin
tokenizer config file saved in test-summarization\checkpoint-43000\tokenizer_config.json
Special tokens file saved in test-summarization\checkpoint-43000\special_tokens_map.json
Deleting older checkpoint [test-summarization\checkpoint-41500] due to args.save_total_limit
Saving model checkpoint to test-summarization\checkpoint-43500
Configuration saved in test-summarization\checkpoint-43500\config.json
Model weights saved in test-summarization\checkpoint-43500\pytorch_model.bin
tokenizer config file saved in test-summarization\checkpoint-43500\tokenizer_config.json
Special tokens file saved in test-summarization\checkpoint-43500\special_tokens_map.json
Deleting older checkpoint [test-summarization\checkpoint-42000] due to args.save_total_limit
Saving model checkpoint to test-summ

The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, article_original, media, extractive, abstractive.
***** Running Evaluation *****
  Num examples = 8561
  Batch size = 2


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=51363, training_loss=1.2181168893340582, metrics={'train_runtime': 9053.6329, 'train_samples_per_second': 11.346, 'train_steps_per_second': 5.673, 'total_flos': 4.194763158144614e+16, 'train_loss': 1.2181168893340582, 'epoch': 3.0})

In [66]:
model.generate

<bound method GenerationMixin.generate of BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, e